In [2]:
import recordlinkage as rl
import pandas as pd
import os
import random as ran
from recordlinkage.preprocessing import *
from prelabeling import *

In [3]:
schema_path = "../schema_matching/csv/schema_final.csv"

In [4]:
schema = pd.read_csv(schema_path, index_col=0)
schema = schema.reset_index(drop=True) # Per evitare che gli indici siano letti come decimali (i.e. 1.0 invece di 1)

/var/folders/c2/dnpzswqd7ll8qj2knwvlk3v80000gn/T/ipykernel_83391/1305971129.py:1: DtypeWarning: Columns (10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  schema = pd.read_csv(schema_path, index_col=0)


# Preprocessamento

## Cleaning dei dati ...

In [5]:
schema_cleaned_path = "./csv/schema_cleaned.csv"
try:
    schema_cleaned = pd.read_csv(schema_cleaned_path, index_col="id")
except:
    schema_cleaned = schema # creo una copia per non modificare lo schema iniziale
    # Cleaning dei dati iterando per riga
    for i in range(len(schema.index)):
        row = schema.iloc[i]
        row_series = row.squeeze() # BOO
        nome = pd.Series(row["name"])
        try:
            nome_cleaned = clean(nome, lowercase=True, replace_by_none='', replace_by_whitespace='\ {2,}', strip_accents='unicode', remove_brackets=False, encoding='utf-8', decode_error='strict')
        except:
            print(nome)
        # Si evita il cleaning di caratteri relative alle valute presenti nei datasets
        row_series_cleaned = clean(row_series, lowercase=True, replace_by_none=r'[^ \\.\\-\\_A-Za-z0-9$€£¥₩₽₹฿₪¤¢₺₱]+', replace_by_whitespace=r'[\\-\\_]', strip_accents="unicode", remove_brackets=False, encoding='utf-8', decode_error='strict')
        schema_cleaned.iloc[i] = row_series_cleaned
        schema_cleaned.iloc[i]["name"] = nome_cleaned[0]
    schema_cleaned.to_csv(schema_cleaned_path, index_label="id")


/var/folders/c2/dnpzswqd7ll8qj2knwvlk3v80000gn/T/ipykernel_83391/1139875684.py:3: DtypeWarning: Columns (10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  schema_cleaned = pd.read_csv(schema_cleaned_path, index_col="id")


# Caricamento dei dataset per classificazione

In [8]:
golden_path_gallo = "./csv/golden_links_gallo.csv"
golden_path_gatto = "./csv/golden_links_gatto.csv"
golden_path_moli = "./csv/golden_links_moli.csv"
golden_path = "./csv/golden_links.csv"

golden_gallo = pd.read_csv(golden_path_gallo, index_col=None)
golden_gatto = pd.read_csv(golden_path_gatto, index_col=None)
golden_moli = pd.read_csv(golden_path_moli, index_col=None)

try:
    golden = pd.read_csv(golden_path, index_col=[0,1])
except:
    golden = pd.concat([golden_gallo, golden_gatto, golden_moli], axis=0)
    golden = golden.reset_index(drop=True)
    golden.to_csv(golden_path, index=None)

In [9]:
golden_country_path_gatto = "./csv/golden_country_links_gatto.csv"
golden_country_path_moli = "./csv/golden_country_links_moli.csv"

golden_country_gatto = pd.read_csv(golden_country_path_gatto, index_col=[0,1])
golden_country_moli = pd.read_csv(golden_country_path_moli, index_col=[0,1])


In [28]:
# golden = pd.concat([golden, golden_country_gatto, golden_country_moli], axis=0)
# golden.to_csv(golden_path)

# Blocking

In [6]:
countryIndexer = rl.Index()
countryIndexer.block(left_on='country', right_on='country')
country_candidate_links = countryIndexer.index(schema_cleaned)
print(country_candidate_links)

MultiIndex([(     1,      0),
            (     3,      0),
            (     3,      1),
            (     4,      0),
            (     4,      1),
            (     4,      3),
            (     5,      0),
            (     5,      1),
            (     5,      3),
            (     5,      4),
            ...
            (159361, 154564),
            (159361, 154690),
            (159361, 154814),
            (159361, 154971),
            (159361, 155014),
            (159361, 155489),
            (159361, 155527),
            (159361, 155691),
            (159361, 157434),
            (159361, 159156)],
           names=['id_1', 'id_2'], length=553892514)


In [7]:
indexer = rl.Index()
indexer.block(left_on='name', right_on='name')
candidate_links = indexer.index(schema_cleaned)
print(candidate_links)

MultiIndex([( 11347,      0),
            ( 18510,      0),
            ( 18510,  11347),
            ( 19510,      0),
            ( 19510,  11347),
            ( 19510,  18510),
            ( 31603,      0),
            ( 31603,  11347),
            ( 31603,  18510),
            ( 31603,  19510),
            ...
            (187476, 152142),
            (162545, 152153),
            (160453, 160452),
            (183776, 172055),
            (183796, 172149),
            (183812, 172198),
            (183656, 172322),
            (183669, 172366),
            (183707, 172428),
            (183813, 172437)],
           names=['id_1', 'id_2'], length=538735)


In [10]:
labeled_country_moli = golden.index.intersection(golden_country_moli.index)
labeled_country_gatto = golden.index.intersection(golden_country_gatto.index)
labeled_country = [*labeled_country_moli,*labeled_country_gatto]

In [11]:
len(candidate_links)
candidate_links = [*candidate_links, *labeled_country]
len(candidate_links)


540835

In [12]:
candidate_links = pd.MultiIndex.from_tuples(candidate_links)

In [28]:
candidate_links

MultiIndex([( 11347,      0),
            ( 18510,      0),
            ( 18510,  11347),
            ( 19510,      0),
            ( 19510,  11347),
            ( 19510,  18510),
            ( 31603,      0),
            ( 31603,  11347),
            ( 31603,  18510),
            ( 31603,  19510),
            ...
            ( 72777,  54402),
            (155831, 103882),
            (103273,  86599),
            (165390, 163151),
            (159765, 118409),
            (166643,  84819),
            (110878,  53982),
            (104008,  59327),
            ( 18308,    248),
            (111728,  97994)],
           length=540835)

# Prepare Labeling

In [6]:
choices_path = "./csv/choices.csv"
# choices_country_path = "./csv/choices_country.csv"

In [ ]:
n_sampled_matches = 30000
try:
    choices = pd.read_csv(choices_path, index_col=[0, 1])
except:
    choices = ran.choices(candidate_links, k=n_sampled_matches)
    choices_df = get_datasetCouples_to_label(choices, schema_cleaned)
    # choices_df.to_csv(choices_path, index=None)

# n_sampled_matches_country = 2100
# try:
#     choices_country = pd.read_csv(choices_country_path, index_col=[0, 1])
# except:
#     choices_country = ran.choices(country_candidate_links, k=n_sampled_matches_country)
#     choices_country_df = get_datasetCouples_to_label(choices_country, schema_cleaned)
#     choices_country_df.to_csv(choices_country_path, index=None)

In [33]:
# choices_path_gallo = "./csv/gallo.csv"
# choices_path_gatto = "./csv/gatto.csv"
# choices_path_moli = "./csv/moli.csv"

# sampled_matches_per_person = n_sampled_matches//3

# choices_df_gallo = choices_df.iloc[:sampled_matches_per_person]
# print(len(choices_df_gallo))

# choices_df_gatto = choices_df.iloc[sampled_matches_per_person:2*sampled_matches_per_person]
# print(len(choices_df_gatto))

# choices_df_moli = choices_df.iloc[2*sampled_matches_per_person:]
# print(len(choices_df_moli))

# choices_df_gallo.to_csv(choices_path_gallo, index=None)
# choices_df_gatto.to_csv(choices_path_gatto, index=None)
# choices_df_moli.to_csv(choices_path_moli, index=None)

10000
10000
10000


## Labeling country

In [10]:
# choices_country_path_gatto = "./csv/gatto_country.csv"
# choices_country_path_moli = "./csv/moli_country.csv"

# sampled_matches_country_per_person = n_sampled_matches_country//2

# choices_country_df_gatto = choices_country_df.iloc[:sampled_matches_country_per_person]
# print(len(choices_country_df_gatto))

# choices_country_df_moli = choices_country_df.iloc[sampled_matches_country_per_person:2*sampled_matches_country_per_person]
# print(len(choices_country_df_moli))

# choices_country_df_gatto.to_csv(choices_country_path_gatto, index=None)
# choices_country_df_moli.to_csv(choices_country_path_moli, index=None)

1050
1050


# Comparing

In [13]:
compare_cl = rl.Compare()
compare_cl.string("name", "name", threshold=0.85, label="name")
compare_cl.string("country", "country", label="country")
compare_cl.string("founded year", "founded year", label="founded year")
compare_cl.string("industry", "industry", label="industry")
compare_cl.string("sector", "sector", label="sector")
compare_cl.string("address", "address", label="address")
compare_cl.string("city", "city", label="city")
compare_cl.string("ceo", "ceo", label="ceo")
features = compare_cl.compute(candidate_links, schema, schema)

In [32]:
features

name   country  founded year  industry  sector  address  city  \
11347  0       0.0  1.000000           0.0       0.0     0.0      0.0   0.0   
18510  0       0.0  0.153846           0.0       0.0     0.0      0.0   0.0   
       11347   1.0  0.153846           0.0       0.0     0.0      0.0   0.0   
19510  0       0.0  1.000000           0.0       0.0     0.0      0.0   0.0   
       11347   1.0  1.000000           0.0       0.0     0.0      0.0   0.0   
...            ...       ...           ...       ...     ...      ...   ...   
166643 84819   0.0  1.000000           0.0       0.0     0.0      0.0   0.0   
110878 53982   0.0  1.000000           0.0       0.0     0.0      0.0   0.0   
104008 59327   0.0  1.000000           0.0       0.0     0.0      0.0   0.0   
18308  248     0.0  1.000000           0.0       0.0     0.0      0.0   0.0   
111728 97994   0.0  1.000000           0.0       0.0     0.0      0.0   0.0   

                   ceo  
11347  0      0.000000  
18510  0      0.421053  
       11347  0.000000  
19510  0      0.000000  
       11347  0.000000  
...                ...  
166643 84819  0.000000  
110878 53982  0.000000  
104008 59327  0.000000  
18308  248    0.000000  
111728 97994  0.000000  

[540835 rows x 8 columns]

In [36]:
features.describe()

,name,country,founded year,industry,sector,address,city,ceo
count,538735.0,538735.000000,538735.000000,538735.000000,538735.000000,538735.000000,538735.000000,538735.000000
mean,1.0,0.437277,0.043698,0.035876,0.043173,0.003701,0.004944,0.050346
std,0.0,0.488800,0.195229,0.177244,0.202804,0.059818,0.070108,0.205419
min,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [37]:
features.sum(axis=1).value_counts().sort_index(ascending=False)

6.000000       461
5.000000       287
4.733333         1
4.685714         1
4.457143         1
             ...  
1.017857         1
1.016667         1
1.016393         1
1.011820         1
1.000000    243056
Length: 5289, dtype: int64

In [33]:
record_linked = features[features.sum(axis=1) > 1]

In [39]:
print(record_linked)

               name   country  founded year  industry  sector  address  city  \
11347  0        1.0  1.000000      0.000000  0.000000     0.0   0.0000   0.0   
18510  0        1.0  0.230769      0.000000  0.000000     0.0   0.0000   0.0   
       11347    1.0  0.230769      0.000000  0.000000     0.0   0.0000   0.0   
19510  0        1.0  1.000000      0.000000  0.000000     0.0   0.0000   0.0   
       11347    1.0  1.000000      0.000000  0.000000     0.0   0.0000   0.0   
...             ...       ...           ...       ...     ...      ...   ...   
148416 144739   1.0  1.000000      0.000000  1.000000     0.0   0.0000   1.0   
148425 144746   1.0  1.000000      0.000000  1.000000     0.0   0.0000   1.0   
152082 145966   1.0  0.000000      0.063492  0.058824     0.0   0.0000   0.0   
151989 151988   1.0  0.000000      1.000000  1.000000     0.0   0.0000   0.0   
160453 160452   1.0  0.000000      0.000000  0.000000     0.0   0.1875   0.0   

                    ceo  
11347  0     

# Train & test split

In [16]:
# Le features relative ai dati etichettati da noi (2125 di cui 1 con label = 0)
company_X = features.loc[golden.index]

In [17]:
# coppie di indici relativi ai dati etichettati con label = 1
company_true_links = golden[golden["label"] == 1].index

In [18]:
# X_train
golden_pairs = company_X[:len(company_X)*3//4]
# y_train
golden_matches_index = golden_pairs.index.intersection(company_true_links)

# Classification (Unsupervised)

In [37]:
ECM = rl.ECMClassifier(binarize=0)
ecm_result = ECM.fit_predict(company_X)
print("Possible record linked size:", len(company_X))
print("Record linked size:", len(ecm_result))
print(f"Removed {len(company_X) - len(ecm_result)} possible links")

Possible record linked size: 4226
Record linked size: 347
Removed 3879 possible links


In [38]:
km = rl.KMeansClassifier()
km_result = km.fit_predict(company_X)
print("Possible record linked size:", len(company_X))
print("Record linked size:", len(km_result))
print(f"Removed {len(company_X) - len(km_result)} possible links")

Possible record linked size: 4226
Record linked size: 164
Removed 4062 possible links


# Evaluation (Unsupervised)

### Confusion matrix

In [39]:
# Expetation conditional maximization
rl.confusion_matrix(company_true_links, ecm_result)

array([[ 226., 1888.],
       [ 121.,   nan]])

In [40]:
# Kmeans
rl.confusion_matrix(company_true_links, km_result)

array([[ 162., 1952.],
       [   2.,   nan]])

### Precision

In [41]:
rl.precision(company_true_links, ecm_result)

0.6512968299711815

In [42]:
rl.precision(company_true_links, km_result)

0.9878048780487805

### Recall

In [43]:
rl.recall(company_true_links, ecm_result)

0.10690633869441817

In [44]:
rl.recall(company_true_links, km_result)

0.07663197729422895

### Accuracy

In [45]:
rl.accuracy(company_true_links, ecm_result, len(record_linked))

0.990153951086427

In [46]:
rl.accuracy(company_true_links, km_result, len(record_linked))

0.9904219039267271

### F-score

In [47]:
rl.fscore(company_true_links, ecm_result)

0.1836651767574157

In [48]:
rl.fscore(company_true_links, km_result)

0.14223002633889376

### Specificity

In [49]:
rl.specificity(company_true_links, ecm_result, len(record_linked))

0.9994043340848898

In [50]:
rl.specificity(company_true_links, km_result, len(record_linked))

0.9999901542823949

# Classification (Supervised)

In [14]:
rlSvm = rl.SVMClassifier()
lr = rl.LogisticRegressionClassifier()
nb = rl.NaiveBayesClassifier(binarize=0.3)

In [19]:
rlSvm.fit(golden_pairs, golden_matches_index)


In [67]:
lr.fit(golden_pairs, golden_matches_index)

In [84]:
nb.fit(golden_pairs, golden_matches_index)

In [68]:
svm_company_pred = rlSvm.predict(company_X)


In [69]:
lr_company_pred = lr.predict(company_X)

In [85]:
nb_company_pred = nb.predict(company_X)

# Evaluation (Supervised)

### Confusion matrix

In [70]:
rl.confusion_matrix(company_true_links, svm_company_pred)

array([[1923.,  191.],
       [ 425.,   nan]])

In [71]:
rl.confusion_matrix(company_true_links, lr_company_pred)

array([[1924.,  190.],
       [ 426.,   nan]])

In [86]:
rl.confusion_matrix(company_true_links, nb_company_pred)

array([[1928.,  186.],
       [ 456.,   nan]])

### Precision

In [72]:
rl.precision(company_true_links, svm_company_pred)

0.8189948892674617

In [73]:
rl.precision(company_true_links, lr_company_pred)

0.8187234042553192

In [87]:
rl.precision(company_true_links, nb_company_pred)

0.8087248322147651

### Recall

In [74]:
rl.recall(company_true_links, svm_company_pred)

0.9096499526963103

In [75]:
rl.recall(company_true_links, lr_company_pred)

0.9101229895931883

In [88]:
rl.recall(company_true_links, nb_company_pred)

0.9120151371807

### Accuracy

In [76]:
rl.accuracy(company_true_links, svm_company_pred, len(company_X))

0.8513961192617132

In [77]:
rl.accuracy(company_true_links, lr_company_pred, len(company_X))

0.8513961192617132

In [89]:
rl.accuracy(company_true_links, nb_company_pred, len(company_X))

0.8452437292948415

### F-score

In [78]:
rl.fscore(company_true_links, svm_company_pred)

0.8619453160017929

In [79]:
rl.fscore(company_true_links, lr_company_pred)

0.8620071684587813

In [90]:
rl.fscore(company_true_links, nb_company_pred)

0.8572698977323253

### Specificity

In [80]:
rl.specificity(company_true_links, svm_company_pred, len(company_X))

0.7976190476190477

In [81]:
rl.specificity(company_true_links, lr_company_pred, len(company_X))

0.7971428571428572

In [91]:
rl.specificity(company_true_links, nb_company_pred, len(company_X))

0.7828571428571428

# Labeling tramite metodi ML

In [20]:
svm_all_matches = rlSvm.predict(features) # gli indici di quelli che matchan

In [21]:
print("All candidates link size:", len(features))
print("All linked pairs by SVM:", len(svm_all_matches))
print("Remove candidate pairs:", len(features)-len(svm_all_matches))

All candidates link size: 540835
All linked pairs by SVM: 494607
Remove candidate pairs: 46228


In [24]:
schema_linked_path = "./csv/schema_linked.csv"
schema_linked = pd.DataFrame(schema_cleaned)
nan = ["nan", "Nan", "NaN", "Not found", "Not Found", "not found", "not Found", "NULL", "null", "Null", "None", "none"]
row_to_drop = set()

# per ogni riga cui indice è in match con un altro
for (idx_l, idx_r) in svm_all_matches:
    # prendere la riga riferita al primo indice, riempire i campi con quelli dell'altra
    # riempio solo le colonne che sono in Nan
    curr_row_l = schema_linked.iloc[idx_l] # riga del primo indice
    curr_row_r = schema_linked.iloc[idx_r] # riga del secondo indice
    for column_data in schema_linked.columns:
        curr_col_l = curr_row_l[column_data]
        if pd.isnull(curr_col_l) or curr_col_l in nan:
            # prendo il campo corrente ma della seconda ennupla
            curr_col_r = curr_row_r[column_data]
            # aggiorno il campo
            schema_linked.iloc[idx_l][column_data] = curr_col_r
    
    # la seconda riga verrà droppata
    row_to_drop.add(idx_r)

# drop di tutte le righe cui campi hanno sostituito quelli delle righe presenti nello schema finale
schema_linked.drop(row_to_drop, inplace=True)

In [25]:
schema_linked

,name,country,market cap,founded year,employees,industry,sector,ceo,revenue,stock,share price,city,address,website
id,,,,,,,,,,,,,,
10682,superit srl,NaN,NaN,NaN,NaN,finance insurance and real estate,NaN,NaN,NaN,NaN,NaN,NaN,via vittor pisani 20 milano 20124 milano italy,NaN
10685,send to me srl,NaN,NaN,NaN,NaN,services,NaN,NaN,NaN,NaN,NaN,NaN,viale ungheria 46 milano 20138 milano italy,NaN
10686,intesa sanpaolo vita spa,NaN,NaN,NaN,NaN,finance insurance and real estate,NaN,NaN,NaN,NaN,NaN,NaN,corso inghilterra 3 torino 10138 torino italy,NaN
10688,totalenergies petrochemicals & refining sa nv,NaN,NaN,NaN,NaN,wholesale trade,NaN,NaN,NaN,NaN,NaN,NaN,via rombon 11 milano 20134 milano italy,NaN
10689,sanastera spa,NaN,NaN,NaN,NaN,finance insurance and real estate,NaN,NaN,NaN,NaN,NaN,NaN,piazza minghetti 4d bologna 40124 bologna italy,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188551,pepsico,usa,$241.05 billion,august 28 1898,309000,beveragesnonalcoholic,consumer staples,mr. ramon luis laguarta,us$70.37 billion 2020,NaN,181,NaN,NaN,httpswww.pepsico.com
188552,costco,usa,$258.15 billion,july 12 1976,288000,discount stores,consumer staples,mr. w. craig jelinek,us$166.76 billion 2020,NaN,483,NaN,NaN,NaN
188553,broadcom,usa,$265.27 billion,1961,20000,semiconductors,technology,mr. hock e. tan,us$23.89 billion 2020,NaN,588,NaN,NaN,httpswww.broadcom.com


In [ ]:
schema_linked.to_csv(schema_linked_path, index_label="id")